In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error
import tqdm

In [2]:
train = pd.read_csv('../input/feedback-prize-english-language-learning/train.csv')
exp154 = np.load('../input/exp154/exp154/preds/oof_pred.npy')
exp159 = np.load('../input/exp159/exp159/preds/oof_pred.npy')
exp160 = np.load('../input/exp160/preds/oof_pred.npy')
exp161 = np.load('../input/exp161/exp161/preds/oof_pred.npy')
exp163 = np.load('/kaggle/input/exp163/exp163/preds/oof_pred.npy')
exp171 = np.load('/kaggle/input/exp171/exp171/preds/oof_pred.npy')
exp170 = np.load('/kaggle/input/exp170/exp170/preds/oof_pred.npy')
exp176 = np.load('/kaggle/input/exp176/exp176/preds/oof_pred.npy')

exp058 = np.load('/kaggle/input/feedback3-exp058-deberta-v3-base/preds/oof_pred.npy')
exp060 = np.load('/kaggle/input/feedback3-exp060-deberta-v3-base/preds/oof_pred.npy')
exp061 = np.load('/kaggle/input/feedback3-exp061-deberta-base/preds/oof_pred.npy')
exp062 = np.load('/kaggle/input/feedback3-exp062-bart-base/preds/oof_pred.npy')
exp063 = np.load('/kaggle/input/feedback3-exp063-lomgformer-base/preds/oof_pred.npy')
exp064 = np.load('/kaggle/input/feedback3-exp065-deberta-v3-base/preds/oof_pred.npy')
exp065 = np.load('/kaggle/input/feedback3-exp065-deberta-v3-base/preds/oof_pred.npy')
exp066 = np.load('/kaggle/input/feedback3-exp066-mup-roberta-large/preds/oof_pred.npy')

In [3]:
team_val_preds = np.stack([exp154, exp159, exp160, exp161, exp163, exp170, exp171, exp176, exp058, exp060, exp061, exp062, exp063, exp064, exp065, exp066], axis=1)

In [4]:
# def metric
target_cols = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']

def mcrmse(y_true,y_pred):
    rmse_scores = []
    for i in range(len(target_cols)):
        rmse_scores.append(np.sqrt(mean_squared_error(y_true[:,i],y_pred[:,i])))
    return np.mean(rmse_scores)

# 損失を定義
def func(weights):
    pred = np.average(team_val_preds, axis=1, weights=weights)
    score = mcrmse(train[target_cols].values, pred)
    return score

model_num = team_val_preds.shape[1]
# 初期値
initial_weights = np.ones(model_num)/model_num
res = minimize(func, initial_weights, method='Nelder-Mead')

# CV, weight
print(res['fun'], res['x'])

np.save('./weight.npy', res['x'])

0.4433801265499658 [ 0.15495818  0.06146405  0.02671482  0.13109993  0.22418948  0.09073126
  0.22163102 -0.0597963   0.05197171  0.0442831  -0.06387709 -0.02850956
 -0.07578908  0.00029019  0.07824899  0.24700227]


In [5]:
fold = pd.read_csv('../input/exp154/exp154/preds/folds.csv')
train['fold'] = fold['0']

In [6]:
# 損失を定義
def func(weights):
    pred = np.average(fold_team_val_preds, axis=1, weights=weights)
    score = mcrmse(train_fold[target_cols].values, pred)
    return score

weights_list = []
initial_weights = np.ones(model_num)/model_num

# 複数回繰り返す
for i in range(15):
    # foldごとに繰り返す
    for fold in range(10):
        train_fold = train.loc[train['fold'] != fold, :]
        fold_team_val_preds = team_val_preds[train_fold.index]
        res = minimize(func, initial_weights, method='Nelder-Mead')

        print("score:", res.fun)
        print("weights:", res.x)
        weights_list.append(res.x)


print(np.array(weights_list).mean(0))

score: 0.4434998009717901
weights: [ 0.14571983  0.06753254  0.03453506  0.13569552  0.18841052  0.07019128
  0.20586173 -0.04806617  0.06297764 -0.00348072 -0.09219247 -0.03194628
 -0.05424149 -0.01901351  0.08817249  0.24051066]
score: 0.44273034051068816
weights: [ 0.09990812  0.03589792  0.00847146  0.07557882  0.16981749  0.0445459
  0.14211999 -0.04463629  0.02911615  0.01377771 -0.04771483 -0.01464841
 -0.00475363  0.11129381 -0.05806193  0.16276454]
score: 0.4426048281475625
weights: [ 0.1736634   0.07493378  0.0276752   0.12440292  0.24985645  0.11759764
  0.19129658 -0.07006751  0.03606989  0.05381963 -0.07352398 -0.02501563
 -0.07407054 -0.0250802   0.08686364  0.24553608]
score: 0.444101571981538
weights: [ 0.1745994   0.04139492  0.02377189  0.14335151  0.21679038  0.08869441
  0.22340312 -0.04800659  0.03824322  0.01546028 -0.04102478 -0.02147977
 -0.10275202  0.26291436 -0.18547383  0.25344552]
score: 0.4427498898404536
weights: [ 0.12988167  0.04646406  0.03094286  0.07

In [7]:
pred = np.average(team_val_preds, axis=1, weights=np.array(weights_list).mean(0))
score = mcrmse(train[target_cols].values, pred)
print(score)

0.4433806443980139


In [8]:
np.save('./moa_weight.npy', np.array(weights_list).mean(0))